## Lab 5 - PID control
Run this cell before running any other cells

In [1260]:
%reload_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

LOG.propagate = False

# Notification Handler
def notification_handler(uuid, byte_array):
    global time_list, dist, pid, pwm, p, i, d
    rx_string = ble.bytearray_to_string(byte_array)
#    print(rx_string)
    if rx_string.find(',') < 0:
        print(rx_string)
    else:
        time, a, b, c, kp, ki, kd = rx_string.split(',')
        time_sec = float(time[2:])/1000
        time_list.append(time_sec)
        dist.append(float(a[3:]))
        pid.append(float(b[3:]))
        pwm.append(float(c[3:]))
        p.append(float(kp[3:]))
        i.append(float(ki[3:]))
        d.append(float(kd[3:]))
        
ble = get_ble_controller()
ble.connect()

while not ble.is_connected():
    pass

ble.start_notify(ble.uuid['RX_STRING'], notification_handler)
ble.send_command(CMD.GET_TIME_MILLIS, "")

2025-04-16 19:08:58,887 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:ec:f2:c2:23:04
2025-04-16 19:09:01,866 | INFO     |: Connected to c0:ec:f2:c2:23:04
T:46750


In [1315]:
# Ku = 0.15-0.2
# Pu = 0.4 sec
# 
# Kp = 0.6 * Ku = .12
# Ki = 2 * Kp / Pu = 0.6
# Kd = Kp * Pu / 8 = 0.006
#ble.send_command(CMD.SET_PID_MAX, "0.12|0.6|0.006|100|25|304|1.1")
#ble.send_command(CMD.SET_PID_MAX, "0.2|0.001|70|140|30|305|1.1") best performance as of 4-16-2025
ble.send_command(CMD.SET_PID_MAX, "0.2|0.001|70|140|30|305|1.1")

KP:0.200 KI:0.001 KD:70.000 PWM_MAX:140 DEADZONE:30 WALL_DIST:305.000 CORRECTION:1.100


In [1316]:
ble.send_command(CMD.GO_WALL, "305")

KP:0.200 KI:0.001 KD:70.000 PWM_MAX:140 DEADZONE:30 CORRECTION:1.200 WALL_DIST:305.000
  Finished Go Wall ... 777
2025-04-16 19:29:56,659 | INFO     |: Disconnected from E8E0F7A4-7773-8749-3CAD-EADBED93F90E


In [1312]:
# Clear the data
time_list = []
dist = []
pid = []
pwm = []
p = []
i = []
d = []

# send the command
ble.send_command(CMD.GET_WALL_DATA, "")

  Sending PID data ...
  Data sent!


In [1]:
#plt.yticks([-180, -120, -60, 0, 60, 120, 180])
count = len(time_list)
total = count - 1
delta = round(time_list[total-1] - time_list[0], 2)

print(count, 'samples in', delta, 'seconds')

plt.figure()
plt.title('Distance (mm)')
plt.xlabel('Time')
#plt.ylabel('Distance (mm)')
plt.axhline(305)
plt.plot(time_list[:total], dist[:total], color='r', label='dist')

plt.figure()
plt.title('PID and PWM')
plt.xlabel('Time')
#plt.ylabel('PID and PWM')
plt.axhline(0)
plt.plot(time_list[10:total],  pid[10:total], color='g', label='pid')
plt.plot(time_list[10:total],  pwm[10:total], color='b', label='pwm')
plt.legend()

plt.figure()
plt.title('PID values')
plt.xlabel('Time')
#plt.ylabel('P, I, D')
plt.axhline(0)
plt.plot(time_list[:total],  p[:total],  color='c', label='P')
plt.plot(time_list[:total],  i[:total],  color='m', label='I')
plt.plot(time_list[:total],  d[:total],  color='y', label='D')

plt.legend()
plt.show()

#for j in range(0, total):
#    print(time_list[j], dist[j], p[j], i[j], d[j], pid[j], pwm[j])

NameError: name 'time_list' is not defined

In [1314]:
# Backward 2 seconds
ble.send_command(CMD.SET_PWM, "-50|3000")

In [1114]:
ble.send_command(CMD.GET_RANGE, 200)

In [148]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

In [147]:
ble.send_command(CMD.SEND_THREE_FLOATS, "1234567890|9876543|0.034567")

In [264]:
# Forward 1 seconds. max speed, then brake hard
ble.send_command(CMD.SET_PWM, "100|0|100|0|1000")
ble.send_command(CMD.SET_PWM, "255|255|255|255|1000")

In [26]:
# Right motor only
ble.send_command(CMD.SET_PWM, "0|0|50|0|2000")

In [27]:
# Left motor only
ble.send_command(CMD.SET_PWM, "50|0|0|0|2000")

In [28]:
# rotate CCW
ble.send_command(CMD.SET_PWM, "1|250|250|1|2000")

In [29]:
# rotate CW
ble.send_command(CMD.SET_PWM, "250|1|1|250|2000")

In [613]:
# Forward 2 seconds
ble.send_command(CMD.SET_PWM, "40|3300")

In [579]:
# stop
ble.send_command(CMD.SET_PWM, "0|0|0|0|0")

In [33]:
# slowest forward
ble.send_command(CMD.SET_PWM, "40|00|40|00|2000")

In [34]:
# slowest reverse
ble.send_command(CMD.SET_PWM, "00|80|00|80|2000")

In [35]:
# slowest turn
ble.send_command(CMD.SET_PWM, "00|80|80|0|2000")

## Disconnect

In [36]:
# Disconnect
ble.disconnect()

2025-02-26 14:10:02,313 | INFO     |: Disconnected from E8E0F7A4-7773-8749-3CAD-EADBED93F90E
